<a href="https://colab.research.google.com/github/SH22Hwang/GNU_AugumentedIntelligenceLab/blob/main/%EC%97%84%EC%A7%80/12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# 8.7 참고
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

print('x_train 데이터 형태:', x_train.shape)
print('x_train[0] 데이터 형태:', x_train[0].shape)

11501568/11490434 [==============================] - 0s 0us/step
x_train 데이터 형태: (60000, 28, 28)
x_train[0] 데이터 형태: (28, 28)


In [ ]:
def plot_images(nRow, nCol, img, cmap='gray'):
  fig = plt.figure()
  fig, ax = plt.subplots(nRow, nCol, figsize = (nCol, nRow))
  for i in range(nRow):
    for j in range(nCol):
      if nRow <= 1: axis = ax[j]
      else: axis = ax[i, j]
      axis.get_xaxis().set_visible(False)
      axis.get_yaxis().set_visible(False)
      axis.imshow(img[i*nCol+j], cmap=cmap)

In [ ]:
from tensorflow.keras import models
from tensorflow.keras import layers

In [ ]:
enc = models.Sequential([layers.Conv2D(filters=16,kernel_size=3, strides=(3,3), activation='elu', input_shape=(28, 28, 1)),
                         layers.Conv2D(filters=8, kernel_size=3, activation='elu'),
                         layers.Flatten(),
                         layers.Dense(1024, activation='elu')])
enc.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 9, 9, 16)          160       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 7, 7, 8)           1160      
_________________________________________________________________
flatten (Flatten)            (None, 392)               0         
_________________________________________________________________
dense (Dense)                (None, 1024)              402432    
Total params: 403,752
Trainable params: 403,752
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# enc = models.Sequential([layers.Conv2D(filters=16,kernel_size=3, strides=(3,3), activation='elu', input_shape=(28, 28, 1)),
#                          layers.Conv2D(filters=16, kernel_size=3, activation='elu'),
#                          layers.Flatten(),
#                          layers.Dense(128, activation='elu')])
# enc.summary()

In [ ]:
dec = models.Sequential([
                         layers.Dense(392, input_shape=(1024, ), activation='elu'),
                         layers.Reshape(target_shape=(7, 7, 8)),
                         layers.Conv2DTranspose(filters=16, kernel_size=3, activation='elu'),
                         layers.Conv2DTranspose(filters=3, kernel_size=3, strides=(3,3), activation='elu')
])
dec.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 392)               401800    
_________________________________________________________________
reshape_1 (Reshape)          (None, 7, 7, 8)           0         
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 9, 9, 16)          1168      
_________________________________________________________________
conv2d_transpose_2 (Conv2DTr (None, 27, 27, 3)         435       
Total params: 403,403
Trainable params: 403,403
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# 인코더, 디코더 결합한 오토 인코더 만들기
AE = models.Sequential([enc, dec])
AE.compile(loss = 'mse')
AE.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential (Sequential)      (None, 1024)              403752    
_________________________________________________________________
sequential_3 (Sequential)    (None, 27, 27, 3)         403403    
Total params: 807,155
Trainable params: 807,155
Non-trainable params: 0
_________________________________________________________________


In [ ]:
X_cnn = x_train.reshape(-1, 27, 27, 3)
history = AE.fit(X_cnn, X_cnn, epochs = 15)
plt.plot(history.history['loss'], 'b-')

ValueError: ignored